In [1]:
import re
import time
import json
import random
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
from concurrent.futures import ThreadPoolExecutor, wait

In [2]:
with open("movies_box.json",'r') as load_f:
    movies_box = json.load(load_f)

In [3]:
wwbo_domain = "http://www.worldwideboxoffice.com/"
for i in range(len(movies_box)):
    movies_box[i]["HREF"]= wwbo_domain + movies_box[i]['HREF'].replace('amp;',"")

In [146]:
movies_box

import concurrent.futures

imdb = []
with concurrent.futures.ThreadPoolExecutor(10) as executor:
    for movie in executor.map(worker, movies_box):
        imdb.append(movie)
    

[{'DOMESTIC': 760.5,
  'HREF': 'http://www.worldwideboxoffice.com/movie.cgi?title=Avatar&year=2009',
  'OVERSEAS': 2027.5,
  'RANK': '0001',
  'TITLES': 'Avatar',
  'WORLD': 2788.0,
  'YEAR': 2009},
 {'DOMESTIC': 659.3,
  'HREF': 'http://www.worldwideboxoffice.com/movie.cgi?title=Titanic&year=1997',
  'OVERSEAS': 1528.1,
  'RANK': '0002',
  'TITLES': 'Titanic',
  'WORLD': 2187.4,
  'YEAR': 1997},
 {'DOMESTIC': 936.0,
  'HREF': 'http://www.worldwideboxoffice.com/movie.cgi?title=Star%20Wars%3A%20Episode%20VII%20%2D%20The%20Force%20Awakens&year=2015',
  'OVERSEAS': 1130.0,
  'RANK': '0003',
  'TITLES': 'Star Wars: Episode VII - The Force Awakens',
  'WORLD': 2066.0,
  'YEAR': 2015},
 {'DOMESTIC': 652.3,
  'HREF': 'http://www.worldwideboxoffice.com/movie.cgi?title=Jurassic%20World&year=2015',
  'OVERSEAS': 1018.1,
  'RANK': '0004',
  'TITLES': 'Jurassic World',
  'WORLD': 1670.4,
  'YEAR': 2015},
 {'DOMESTIC': 623.4,
  'HREF': 'http://www.worldwideboxoffice.com/movie.cgi?title=The%20Avenge

In [4]:
def ipGet():
    req = r.get("https://free-proxy-list.net/")
    resList = bs(req.text, "lxml").select_one(".table-striped").select("tbody tr")
    ipList = [resList[i].select("td")[0].text + ":" + resList[i].select("td")[1].text
              for i in range(len(resList))]
    return {"http": "http://" + random.choice(ipList),"https": "https://" + random.choice(ipList)}

In [5]:
tStart = time.time()
counter = 1
proxies = ipGet() 
for i in range(len(movies_box)):
    #print(movies_box[i]["HREF"])
    movie_url = movies_box[i]['HREF']
    #movie_url = url
    movie_resp = r.get(movie_url,proxies=proxies)
    movie_soup = bs(movie_resp.text, 'html5lib')
    imdb_url = [x.get("href") for x in movie_soup.select("td > a") if x.get("href").startswith("http://www.imdb.com/")][0]
    print(str(counter) + "," + imdb_url)
    imdb_resp = r.get(imdb_url,proxies=proxies)
    imdb_soup = bs(imdb_resp.text,"html5lib")
    try:
        movie_number = ([x.get("href") for x in imdb_soup.select("head > link") if x.get("href").startswith("http://www.imdb.com/")][0]).split('/')[-2]
        movies_box[i]["MOVIE_NUMBER"] = movie_number
    except:
        continue
    imdb_domain = "http://www.imdb.com"
    imdb_plot_url = imdb_domain + '/title/' + movies_box[i]["MOVIE_NUMBER"] + '/plotsummary?ref_=tt_stry_pl'
    imdb_plot_resp = r.get(imdb_plot_url,proxies=proxies)
    imdb_plot_soup = bs(imdb_plot_resp.text,"html5lib")
    imdb_plot_Summaries = imdb_plot_soup.select("section > ul#plot-summaries-content")[0].text.replace("\n","").strip()
    imdb_plot_Synopsis = imdb_plot_soup.select("section > ul#plot-synopsis-content")[0].text.replace("\n","").strip()
    movies_box[i]["Summaries"] = imdb_plot_Summaries
    movies_box[i]["Synopsis"] = imdb_plot_Synopsis
    
    imdb_Certi_url = imdb_domain +'/title/' +movies_box[i]["MOVIE_NUMBER"] + '/parentalguide?ref_=tt_stry_pg'
    imdb_Certi_resp = r.get(imdb_Certi_url,proxies=proxies)
    imdb_Certi_soup = bs(imdb_Certi_resp.text,"html5lib")
    imdb_Certi_soup.select("#certifications-list > td > ul > li > a")
    try:
        imdb_Certification_US = [x.get("href") for x in imdb_Certi_soup.select("#certifications-list > td > ul > li > a") \
                             if x.get("href").startswith("/search/title?certificates=US")][0].split(":")[-1]
        movies_box[i]["Certification"] = "US" + " " + imdb_Certification_US
    except:
        try:
            imdb_Certification_HK = [x.get("href") for x in imdb_Certi_soup.select("#certifications-list > td > ul > li > a") \
                             if x.get("href").startswith("/search/title?certificates=HK")][0].split(":")[-1]
            movies_box[i]["Certification"] = "HK" + " " + imdb_Certification_HK
        except:
            movies_box[i]["Certification"] = 'other_country'
    
    imdb_Release_url =  imdb_domain +'/title/' +movies_box[i]["MOVIE_NUMBER"] + '/releaseinfo?ref_=tt_dt_dt'
    imdb_Release_resp = r.get(imdb_Release_url,proxies=proxies)
    imdb_Release_soup = bs(imdb_Release_resp.text,"html5lib")
    imdb_Release_Date = imdb_Release_soup.select("#release_dates > tbody > tr > td.release_date")[0].text
    imdb_Release_Country = imdb_Release_soup.select("#release_dates > tbody > tr > td > a")[0].text
    movies_box[i]["Date"] = imdb_Release_Date
    movies_box[i]["Country"] = imdb_Release_Country
    
    try:
        imdb_Awards = imdb_soup.select("div > div#titleAwardsRanks > span")[0].text.replace("\n","").strip()
        movies_box[i]["Awards"] = imdb_Awards
    except:
        movies_box[i]["Awards"] = "None"
    
    imdb_Production_Co = []
    for co_num in range(len(imdb_soup.select("div#titleDetails > div > span > a > span.itemprop"))):
        imdb_Production_Co.append(imdb_soup.select("div#titleDetails > div > span > a > span.itemprop")[co_num].text)
    movies_box[i]["Production_Co"] = imdb_Production_Co
    counter +=1

with open("IMDB.json", "w") as f:
    jd = json.dumps(movies_box, ensure_ascii=False, indent=4)
    jd.encode("utf-8")
    f.write(jd)
tEnd = time.time()
print(tEnd-tStart)

1,http://www.imdb.com/Business?Avatar%20(2009)
2,http://www.imdb.com/Business?Titanic%20(1997)
3,http://www.imdb.com/Business?Star%20Wars%3A%20Episode%20VII%20%2D%20The%20Force%20Awakens%20(2015)
4,http://www.imdb.com/Business?Jurassic%20World%20(2015)
5,http://www.imdb.com/Business?The%20Avengers%20(2012)
6,http://www.imdb.com/Business?Furious%20Seven%20(2015)
7,http://www.imdb.com/Business?Avengers%3A%20Age%20of%20Ultron%20(2015)
8,http://www.imdb.com/Business?The%20Fate%20of%20the%20Furious%20(2017)
9,http://www.imdb.com/Business?Harry%20Potter%20and%20the%20Deathly%20Hallows%3A%20Part%202%20(2011)
10,http://www.imdb.com/Business?Frozen%20(2013%2FI)
11,http://www.imdb.com/Business?Beauty%20and%20the%20Beast%20(2017)
12,http://www.imdb.com/Business?Iron%20Man%20Three%20(2013)
13,http://www.imdb.com/Business?Minions%20(2015)
14,http://www.imdb.com/Business?Captain%20America%3A%20Civil%20War%20(2016)
15,http://www.imdb.com/Business?Transformers%3A%20Dark%20of%20the%20Moon%20(2011)
16,h

113,http://www.imdb.com/Business?Star%20Wars%20(1977)
114,http://www.imdb.com/Business?Star%20Wars%3A%20Episode%20II%20%2D%20Attack%20of%20the%20Clones%20(2002)
115,http://www.imdb.com/Business?Thor%3A%20The%20Dark%20World%20(2013)
116,http://www.imdb.com/Business?Moana%20(2016%2FI)
117,http://www.imdb.com/Business?Kung%20Fu%20Panda%20(2008)
118,http://www.imdb.com/Business?Sing%20(2016)
119,http://www.imdb.com/Business?The%20Incredibles%20(2004)
120,http://www.imdb.com/Business?The%20Martian%20(2015)
121,http://www.imdb.com/Business?Fast%20Five%20(2011)
122,http://www.imdb.com/Business?Hancock%20(2008)
123,http://www.imdb.com/Business?Men%20in%20Black%203%20(2012)
124,http://www.imdb.com/Business?Iron%20Man%202%20(2010)
125,http://www.imdb.com/Business?Ratatouille%20(2007)
126,http://www.imdb.com/Business?How%20to%20Train%20Your%20Dragon%202%20(2014)
127,http://www.imdb.com/Business?The%20Lost%20World%3A%20Jurassic%20Park%20(1997)
128,http://www.imdb.com/Business?Logan%20(2017)
129,ht

240,http://www.imdb.com/Business?The%20Wolverine%20(2013)
241,http://www.imdb.com/Business?Kingsman%3A%20The%20Secret%20Service%20(2014)
242,http://www.imdb.com/Business?Night%20at%20the%20Museum%3A%20Battle%20of%20the%20Smithsonian%20(2009)
243,http://www.imdb.com/Business?Batman%20(1989)
244,http://www.imdb.com/Business?Pacific%20Rim%20(2013)
245,http://www.imdb.com/Business?The%20Bodyguard%20(1992)
246,http://www.imdb.com/Business?Ice%20Age%3A%20Collision%20Course%20(2016)
247,http://www.imdb.com/Business?Signs%20(2002)
248,http://www.imdb.com/Business?X2%20(2003)
249,http://www.imdb.com/Business?The%20Mummy%20(2017)
250,http://www.imdb.com/Business?Prometheus%20(2012%2FI)
251,http://www.imdb.com/Business?The%20Mummy%3A%20Tomb%20of%20the%20Dragon%20Emperor%20(2008)
252,http://www.imdb.com/Business?Gone%20with%20the%20Wind%20(1939)
253,http://www.imdb.com/Business?Tron%20(2010)
254,http://www.imdb.com/Business?Mission%3A%20Impossible%20III%20(2006)
255,http://www.imdb.com/Business?Sn

ProxyError: HTTPConnectionPool(host='139.162.70.177', port=51089): Max retries exceeded with url: http://www.imdb.com/title/tt3110958/parentalguide?ref_=tt_stry_pg (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5d691c9c88>: Failed to establish a new connection: [Errno 111] Connection refused',)))

In [27]:
imdb_url

'http://www.imdb.com/Business?WALL%C2%B7E%20(2008)'

In [ ]:
# def worker(url):
#     url = 

# numThread = 80
# threads = ThreadPoolExecutor(numThread)
# futures = []
# for url in urls:
#     futures.append(threads.submit(worker, url))
# wait(futures)

In [12]:
# domain = "http://www.worldwideboxoffice.com/"
# for i in range(len(movies_box)):
#     movies_box[i]["url"]= domain + movies_box[i]['HREF']
#     print(movies_box[i])

{'RANK': '0001', 'DOMESTIC': 760.5, 'OVERSEAS': 2027.5, 'WORLD': 2788.0, 'TITLES': 'Avatar', 'YEAR': 2009, 'HREF': 'movie.cgi?title=Avatar&amp;year=2009', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Avatar&amp;year=2009'}
{'RANK': '0002', 'DOMESTIC': 659.3, 'OVERSEAS': 1528.1, 'WORLD': 2187.4, 'TITLES': 'Titanic', 'YEAR': 1997, 'HREF': 'movie.cgi?title=Titanic&amp;year=1997', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Titanic&amp;year=1997'}
{'RANK': '0003', 'DOMESTIC': 936.0, 'OVERSEAS': 1130.0, 'WORLD': 2066.0, 'TITLES': 'Star Wars: Episode VII - The Force Awakens', 'YEAR': 2015, 'HREF': 'movie.cgi?title=Star%20Wars%3A%20Episode%20VII%20%2D%20The%20Force%20Awakens&amp;year=2015', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Star%20Wars%3A%20Episode%20VII%20%2D%20The%20Force%20Awakens&amp;year=2015'}
{'RANK': '0004', 'DOMESTIC': 652.3, 'OVERSEAS': 1018.1, 'WORLD': 1670.4, 'TITLES': 'Jurassic World', 'YEAR': 2015, 'HREF': 'movie.cgi?title=Juras

{'RANK': '<b><font color="#17FF17">1539', 'DOMESTIC': 7.1, 'OVERSEAS': 91.8, 'WORLD': 98.9, 'TITLES': 'Beatriz at Dinner', 'YEAR': 2017, 'HREF': 'movie.cgi?title=Beatriz%20at%20Dinner&amp;year=2017', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Beatriz%20at%20Dinner&amp;year=2017'}
{'RANK': '1540', 'DOMESTIC': 0.7, 'OVERSEAS': 98.1, 'WORLD': 98.8, 'TITLES': 'Les visiteurs', 'YEAR': 1993, 'HREF': 'movie.cgi?title=Les%20visiteurs&amp;year=1993', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Les%20visiteurs&amp;year=1993'}
{'RANK': '1541', 'DOMESTIC': 0.1, 'OVERSEAS': 98.7, 'WORLD': 98.8, 'TITLES': 'Di Renjie: Shen du long wang', 'YEAR': 2013, 'HREF': 'movie.cgi?title=Di%20Renjie%3A%20Shen%20du%20long%20wang&amp;year=2013', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Di%20Renjie%3A%20Shen%20du%20long%20wang&amp;year=2013'}
{'RANK': '1542', 'DOMESTIC': 66.5, 'OVERSEAS': 32.2, 'WORLD': 98.7, 'TITLES': 'Bowfinger', 'YEAR': 1999, 'HREF': 'movie.cgi?title

{'RANK': '4224', 'DOMESTIC': 18.7, 'OVERSEAS': 18.7, 'WORLD': '0', 'TITLES': 'Ringu 2', 'YEAR': 1999, 'HREF': 'movie.cgi?title=Ringu%202&amp;year=1999', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Ringu%202&amp;year=1999'}
{'RANK': '4225', 'DOMESTIC': 18.7, 'OVERSEAS': 18.7, 'WORLD': '0', 'TITLES': 'Casualties of War', 'YEAR': 1989, 'HREF': 'movie.cgi?title=Casualties%20of%20War&amp;year=1989', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Casualties%20of%20War&amp;year=1989'}
{'RANK': '4226', 'DOMESTIC': 11.9, 'OVERSEAS': 6.8, 'WORLD': 18.7, 'TITLES': 'Glee: The 3D Concert Movie', 'YEAR': 2011, 'HREF': 'movie.cgi?title=Glee%3A%20The%203D%20Concert%20Movie&amp;year=2011', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Glee%3A%20The%203D%20Concert%20Movie&amp;year=2011'}
{'RANK': '4227', 'DOMESTIC': 18.7, 'OVERSEAS': 18.7, 'WORLD': '0', 'TITLES': 'Anywhere But Here', 'YEAR': 1999, 'HREF': 'movie.cgi?title=Anywhere%20But%20Here&amp;year=1999', 'url': 

{'RANK': '6914', 'DOMESTIC': 4.2, 'OVERSEAS': 4.2, 'WORLD': '0', 'TITLES': 'Exotica', 'YEAR': 1994, 'HREF': 'movie.cgi?title=Exotica&amp;year=1994', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Exotica&amp;year=1994'}
{'RANK': '6915', 'DOMESTIC': 4.0, 'OVERSEAS': 0.2, 'WORLD': 4.2, 'TITLES': 'Peaceful Warrior', 'YEAR': 2006, 'HREF': 'movie.cgi?title=Peaceful%20Warrior&amp;year=2006', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Peaceful%20Warrior&amp;year=2006'}
{'RANK': '6916', 'DOMESTIC': 2.0, 'OVERSEAS': 2.2, 'WORLD': 4.2, 'TITLES': 'Betrayed', 'YEAR': 1954, 'HREF': 'movie.cgi?title=Betrayed&amp;year=1954', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Betrayed&amp;year=1954'}
{'RANK': '6917', 'DOMESTIC': 4.2, 'OVERSEAS': 4.2, 'WORLD': '0', 'TITLES': 'Priest', 'YEAR': 1994, 'HREF': 'movie.cgi?title=Priest&amp;year=1994', 'url': 'http://www.worldwideboxoffice.com/movie.cgi?title=Priest&amp;year=1994'}
{'RANK': '6918', 'DOMESTIC': 4.2, 'OVERSEAS':

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [3]:
# wwbo_domain = "http://www.worldwideboxoffice.com/"
# movies_box[0]["HREF"]= wwbo_domain + movies_box[0]['HREF'].replace('amp;',"")
movie_url = movies_box[0]['HREF']
movie_resp = r.get(movie_url)
movie_soup = bs(movie_resp.text, 'html5lib')
movie_soup


<html><head>
<title>Avatar (2009) </title>
<meta content="WorldwideBoxoffice.com" name="index"/>
<meta content="Avatar (2009) [ $760,507,625 (domestic) + $2,027,457,460 (overseas) = $2,787,965,085 (worldwideboxoffice.com) ]" name="description"/>
<link href="http://www.worldwideboxoffice.com/favicon.ico" rel="SHORTCUT ICON"/>
<link href="http://www.worldwideboxoffice.com/movie.cgi?title=Avatar&amp;year=2009" rel="canonical"/>
</head>

<body alink="#FFFF00" bgcolor="#000044" link="#00eeff" text="#FFFFFF" vlink="ffff88">
<center>

<table>
<tbody><tr>
<td align="CENTER" bgcolor="#00ccff" border="0" cellpadding="5" cellspacing="5" valign="TOP" width="100%">
<h1><font color="#000055" size="6">
WorldwideBoxoffice 
<font size="4">(in millions of U.S. dollars)

</font></font></h1>
<p>

</p><form action="index.cgi" method="GET">
<font color="#000055" size="6"><font size="4"><font color="#000055">
<input type="submit" value="List movies"/> by 
<select name="order">
<option> domestic
</option><opt

In [154]:
imdb_url = [x.get("href") for x in movie_soup.select("td > a") if x.get("href").startswith("http://www.imdb.com/")][0]
imdb_resp = r.get(imdb_url)
imdb_soup = bs(imdb_resp.text,"html5lib")
movie_url

'http://www.worldwideboxoffice.com/movie.cgi?title=Avatar&year=2009'

In [5]:
movie_number = ([x.get("href") for x in imdb_soup.select("head > link") if x.get("href").startswith("http://www.imdb.com/")][0]).split('/')[-2]
movies_box[0]["MOVIE_NUMBER"] = movie_number
movies_box[0]

{'DOMESTIC': 760.5,
 'HREF': 'http://www.worldwideboxoffice.com/movie.cgi?title=Avatar&year=2009',
 'MOVIE_NUMBER': 'tt0499549',
 'OVERSEAS': 2027.5,
 'RANK': '0001',
 'TITLES': 'Avatar',
 'WORLD': 2788.0,
 'YEAR': 2009}

In [6]:
imdb_domain = "http://www.imdb.com"
imdb_plot = [x.get("href") for x in imdb_soup.select(".article > .see-more > a") if x.get("href").startswith("/title/")][2]
imdb_plot_url = imdb_domain + imdb_plot
imdb_plot_resp = r.get(imdb_plot_url)
imdb_plot_soup = bs(imdb_plot_resp.text,"html5lib")
imdb_plot_soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head>
        <meta charset="utf-8"/>
        <meta content="IE=edge" http-equiv="X-UA-Compatible"/>

    <meta content="app-id=342792525, app-argument=imdb:///title/tt0499549?src=mdot" name="apple-itunes-app"/>
            <script type="text/javascript">var ue_t0=window.ue_t0||+new Date();</script>
            <script type="text/javascript">
                var ue_mid = "A1EVAM02EL8SFB"; 
                var ue_sn = "www.imdb.com";  
                var ue_furl = "fls-na.amazon.com";
                var ue_sid = "000-0000000-0000000";
                var ue_id = "1GMZXY51FF47TYGSG8WH";
                (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:argu

In [7]:
imdb_plot_Summaries = imdb_plot_soup.select("section > ul#plot-summaries-content")[0].text.replace("\n","").strip()
imdb_plot_Synopsis = imdb_plot_soup.select("section > ul#plot-synopsis-content")[0].text.replace("\n","").strip()

In [8]:
imdb_Certi_url = imdb_domain +'/title/' +movies_box[0]["MOVIE_NUMBER"] + '/parentalguide?ref_=tt_stry_pg'
imdb_Certi_resp = r.get(imdb_Certi_url)
imdb_Certi_soup = bs(imdb_Certi_resp.text,"html5lib")
imdb_Certi_soup.select("#certifications-list > td > ul > li > a")
imdb_Certification_US = [x.get("href") for x in imdb_Certi_soup.select("#certifications-list > td > ul > li > a") \
                         if x.get("href").startswith("/search/title?certificates=US")][0].split(":")[-1]

In [9]:
imdb_Certification_US

'PG-13'

In [10]:
imdb_Release_url =  imdb_domain +'/title/' +movies_box[0]["MOVIE_NUMBER"] + '/releaseinfo?ref_=tt_dt_dt'
imdb_Release_resp = r.get(imdb_Release_url)
imdb_Release_soup = bs(imdb_Release_resp.text,"html5lib")
#[x.get('href') for x in imdb_Release_soup.select("#release_dates > tbody > tr > td > a") if x.get('href').startswith("/calendar/?region=us")]
imdb_Release_Date = imdb_Release_soup.select("#release_dates > tbody > tr > td.release_date")[0].text
imdb_Release_Country = imdb_Release_soup.select("#release_dates > tbody > tr > td > a")[0].text
imdb_Release_Country

'UK'

In [11]:
imdb_Release_Date

'10 December 2009'

In [57]:
imdb_Awards = imdb_soup.select("div > div#titleAwardsRanks > span")[0].text.replace("\n","").strip()

In [92]:
#DON'T USE. It's unpredictable.
#imdb_Budget = imdb_soup.select("#titleDetails > div")[6].text.split(":")[1].split("\n")[0]
#imdb_Opening_Weekend = imdb_soup.select("#titleDetails > div")[7].text.split(":")[1].split("\n")

In [138]:
imdb_Production_Co = []
for co_num in range(len(imdb_soup.select("div#titleDetails > div > span > a > span.itemprop"))):
    imdb_Production_Co.append(imdb_soup.select("div#titleDetails > div > span > a > span.itemprop")[co_num].text)
imdb_Production_Co

['Twentieth Century Fox', 'Dune Entertainment', 'Ingenious Film Partners']